In [ ]:
def lensort(a):
    n = len(a)
    for i in range(n):
        for j in range(i+1,n):
            if len(a[i]) < len(a[j]):
                temp = a[i]
                a[i] = a[j]
                a[j] = temp
    return a

f = open("D:/Toki_1805030/Dna_protein_interaction/DNA_PROTEIN-20220506T153412Z-002/DNA_PROTEIN/train.txt", "r")
count=0
seq_train=[]
for x in f:
    if(count%2==0):
      label=x.split('|')[1]





    if(count%2==1):
      x=str(label)+x
      x=x.replace('\n','')
      x=" ".join(x)
      seq_train.append(x)

    count+=1

sorted_train=lensort(seq_train)
max_seq_len=2650
f = open("D:/Toki_1805030/Dna_protein_interaction/DNA_PROTEIN-20220506T153412Z-002/DNA_PROTEIN/PDB_test.txt", "r")
count=0
seq_test=[]
for x in f:
    if(count%2==0):
      label=x.split('|')[1]





    if(count%2==1):
      x=str(label)+x
      x=x.replace('\n','')
      x=" ".join(x)
      seq_test.append(x)

    count+=1

processed_proteins=[]
count=0

for protein in seq_test:


  if(len(protein)<=max_seq_len):

    processed_proteins.append(protein)

  else:
    #print(count)
    label=protein[0]
    protein=protein[1:]
    i=0
    length=len(protein)
    while(i<=length):
      temp=label+' '
      temp=label+protein[i:i+max_seq_len]

      processed_proteins.append(temp)

      i=i+max_seq_len


sorted_test=lensort(processed_proteins)
import numpy as np
label_train=[]
for prot in sorted_train:
  label_train.append(int(prot[0]))


label_test=[]
for prot in sorted_test:
  label_test.append(int(prot[0]))

for i in range(len(sorted_train)):
  sorted_train[i]=sorted_train[i][1:]

for i in range(len(sorted_test)):
  sorted_test[i]=sorted_test[i][1:]

train=np.asarray(label_train)-1
test=np.asarray(label_test)-1

In [ ]:
sorted_test=lensort(seq_test)
len(sorted_test[-1])

103

In [ ]:
np.save('D:/Toki_1805030/Dna_protein_interaction/DNA_PROTEIN-20220506T153412Z-002/DNA_PROTEIN/train.npy',train)
np.save('D:/Toki_1805030/Dna_protein_interaction/DNA_PROTEIN-20220506T153412Z-002/DNA_PROTEIN/PDB_test.npy',test)

In [ ]:
!pip install -q SentencePiece transformers

     |████████████████████████████████| 1.3 MB 6.4 MB/s 
     |████████████████████████████████| 5.5 MB 51.3 MB/s 
     |████████████████████████████████| 163 kB 73.9 MB/s 
     |████████████████████████████████| 7.6 MB 67.8 MB/s 


In [ ]:
import torch
from transformers import T5EncoderModel, T5Tokenizer
import re
import numpy as np
import gc

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("Rostlab/prot_t5_xl_uniref50", do_lower_case=False )

Downloading:   0%|          | 0.00/238k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

In [ ]:
model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_uniref50",map_location=torch.device('cpu'))

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/11.3G [00:00<?, ?B/s]

TypeError: ignored

In [ ]:
# torch.save(model, r'D:\Toki_1805030\Dna_protein_interaction\DNA_PROTEIN-20220506T153412Z-002\DNA_PROTEIN\protbert_x_50')

In [ ]:
gc.collect()

49

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model = model.eval()

In [ ]:
sequences_Example = ["A E T C Z A O","S K T Z P"]
sequences_Example = [re.sub(r"[UZOB]", "X", sequence) for sequence in sequences_Example]

In [ ]:
ids = tokenizer.batch_encode_plus(sequences_Example, add_special_tokens=True, padding=True)
input_ids = torch.tensor(ids['input_ids']).to(device)
attention_mask = torch.tensor(ids['attention_mask']).to(device)

In [ ]:
with torch.no_grad():
    embedding = model(input_ids=input_ids,attention_mask=attention_mask)
embedding = embedding.last_hidden_state.cpu().numpy()

In [ ]:
features = []
for seq_num in range(len(embedding)):
    seq_len = (attention_mask[seq_num] == 1).sum()
    seq_emd = embedding[seq_num][:seq_len-1]
    features.append(seq_emd)

In [ ]:
features[0].shape


(7, 1024)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
torch.cuda.memory_allocated(0)/1e9

5.734095872

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
all_protein_features = []

for i, seq in enumerate(sorted_test):

    print(i)

    ids = tokenizer.batch_encode_plus([seq], add_special_tokens=True, pad_to_max_length=False)

    input_ids = torch.tensor(ids['input_ids']).to(device)


    #attention_mask = torch.tensor(ids['attention_mask']).to(device)

    print(input_ids.shape)
    with torch.no_grad():
        print("hi")
        embedding = model(input_ids=input_ids[:500],attention_mask=None)[0]
        print("hi2")

    embedding = embedding.cpu().numpy()
    #print(embedding.shape)

    features = [embedding]
    '''for seq_num in range(len(embedding)):
        seq_len = (attention_mask[seq_num] == 1).sum()
        seq_emd = embedding[seq_num][1:seq_len-1]
        features.append(seq_emd)
        print(len(features[0]))'''

    all_protein_features += features



0
torch.Size([1, 5185])
hi


RuntimeError: CUDA out of memory. Tried to allocate 3.21 GiB (GPU 0; 8.00 GiB total capacity; 5.34 GiB already allocated; 468.62 MiB free; 5.35 GiB reserved in total by PyTorch)

In [ ]:
import pickle
import gzip
pickle.dump({'ProtBert_features':all_protein_features},
            gzip.open('D:/Toki_1805030/Dna_protein_interaction/DNA_PROTEIN-20220506T153412Z-002/DNA_PROTEIN/Protx50_features_test.pkl.gz',
                        'wb')
                        
        )